### 1. Prepare the working environment

In [ ]:
HOME = "../.." # path to the root of the project two levels up

# Activate the environment
using Pkg
Pkg.activate(HOME)
Pkg.precompile()

In [ ]:
using IceFloeTracker: pairfloes, deserialize, PaddedView, float64, mosaicview, Gray
using DataFrames
imshow(x) = Gray.(x);


### 2. Set thresholds for the algorithm

In [ ]:
# Set thresholds
t1 = (dt=(30.0, 100.0, 1300.0), dist=(200, 250, 300))
t2 = (
    area=1200,
    arearatio=0.28,
    majaxisratio=0.10,
    minaxisratio=0.12,
    convexarearatio=0.14,
)
t3 = (
    area=10_000,
    arearatio=0.18,
    majaxisratio=0.1,
    minaxisratio=0.15,
    convexarearatio=0.2,
)
condition_thresholds = (t1, t2, t3)
mc_thresholds = (
    goodness=(area3=0.18, area2=0.236, corr=0.68), comp=(mxrot=10, sz=16)
)

### 3. Load data

In [ ]:
# Load data
pth = joinpath(HOME, "test", "test_inputs", "tracker")
floedata = deserialize(joinpath(pth,"tracker_test_data.dat"))
passtimes = deserialize(joinpath(pth,"passtimes.dat"))

latlonimgpth = joinpath(HOME, "test", "test_inputs", "NE_Greenland_truecolor.2020162.aqua.250m.tiff")

props, imgs = deepcopy(floedata.props), deepcopy(floedata.imgs);

# Filter out floes with area less than 350 pixels
for (i, prop) in enumerate(props)
    props[i] = prop[prop[:, :area].>=350, :]
end

### 4. Pair and label floes

In [ ]:
# Get paired floes with labels; other tracking data is in the `tracked` variable
pairs = pairfloes(imgs, props, passtimes, latlonimgpth, condition_thresholds, mc_thresholds)